### 4. 1st-order Wasserstein distance

Let $X \in \mathcal{X}:=\{-2,0,2\}$ be a discrete random variable with probability distribution: $\mathbb{P}_X(x)=\frac{1}{4}, \frac{1}{2}, \frac{1}{4}$ for $x=-2,0,2$, respectively. Let $Y \in \mathcal{Y}:=\{-4,-1,1,4\}$ be another discrete random variable with: $\mathbb{P}_Y(y)=\frac{3}{8}, \frac{1}{8}, \frac{1}{8}, \frac{3}{8}$ for $y=-4,-1,1,4$, respectively. Consider Monge's problem which can be formulated as follows. Given $\mathbb{P}_X$ and $\mathbb{P}_Y$,

$$
W\left(\mathbb{P}_X, \mathbb{P}_Y\right):=\min _{\mathbb{P}_{X, Y}} \mathbb{E}[\|X-Y\|]
$$

where the minimization is over all joint distributions $\mathbb{P}_{X, Y}$ respecting the marginals $\mathbb{P}_X$ and $\mathbb{P}_Y$ :

$$
\begin{array}{ll}
\sum_{y \in \mathcal{Y}} \mathbb{P}_{X, Y}(x, y)=\mathbb{P}_X(x) & \forall x \in \mathcal{X} \\
\sum_{x \in \mathcal{X}} \mathbb{P}_{X, Y}(x, y)=\mathbb{P}_Y(y) & \forall y \in \mathcal{Y}
\end{array}
$$


(b) Solve the optimization problem using CVXPY.

Hint: Given an optimization variable, say $z \in \mathbf{R}^d$, and $(w, A, b, C, e)$ that appear in the LP standard form, here is a sample script:

```python

import cvxpy as cp
import numpy as np

z = cp.Variable(d)
objective = cp.Minimize(np.transpose(w) @ z)
constraints = [A@z-b <= 0, C@z-e == 0]
prob = cp.Problem(objective, constraints)
prob.solve()

print("status: ", prob.status)
print("Optimal value: ", prob.value)
print("Optimal var: ", z.value)

```


### 5. Linear classification

Consider the linear classification problem wherein the goal is to find a boundary of the line form that can distinguish legitimate emails from spams. We are given $\left\{\left(x^{(i)}, y^{(i)}\right)\right\}_{i=1}^m$ training dataset (given in the file "train.csv" uploaded on the course website). Here $x^{(i)}:=\left(x_1^{(i)}, x_2^{(i)}\right)$ indicates a feature vector of the $i$ th example and $y^{(i)}$ denotes its corresponding label (legitimate $=+1$; spam $=-1$ ).


(a) What are $m$ and dimension of $x^{(i)}$ ?

Hint: You may want to use the following script:
```python 

import pandas as pd

data = pd.read_csv(’train.csv’)
X = data[['X1', 'X2']].values
y = data['Y'].values
m,n = X.shape
print(m,n)

```

(b) Use a Python code below to visualize the data points in the two-dimensional space.

```python 
import matplotlib.pyplot as plt

X_y1 = X[y==+1] # legitimate
X_y0 = X[y==-1] # spam

plt.figure(figsize=(4,4), dpi=150)

plt.scatter(X_y1[:,0],X_y1[:,1],
            c=’blue’, label = ’legitimate’, marker=’o’)
plt.scatter(X_y0[:,0],X_y0[:,1],
            c=’red’, label = ’spam’, marker=’o’)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.title('Visualization of data points')
plt.show()
```

(c) In class, we formulated a margin-based linear classifier as:

$$
\begin{aligned}
& \min _{a, b, v^{(i)}} \sum_{i=1}^m v(i): \\
& \quad y^{(i)}\left(a^T x^{(i)}-b\right)+v^{(i)} \geq 1, \quad i \in\{1, \ldots, m\} \\
& \quad v^{(i)} \geq 0
\end{aligned}
$$

Using a code below, print out the optimal solution.

```python 
import cvxpy as cp

a = cp.Variable(n)
b = cp.Variable()
v = cp.Variable(m)

# constraints and the objective function
constraints = [y[i]*(X[i,:]@a-b) + v[i]>=1 for i in range(m)] + [v>=0]
objective = cp.Minimize(cp.sum(v))

prob = cp.Problem(objective, constraints)
prob.solve()

print('status: ', prob.status)
print('optimal_value: ', prob.value)
print('optimal_var: ')

print('a*: ',a.value)
print('b*: ',b.value)
print('v*: ',v.value)

```

### 7. Margin-based linear classifier vs. Least-Squares classifier 

Consider the legitimate-vs-spam email classification. We are given $\{(x^{(i)}, y^{(i)})\}^m_{i=1}$ training dataset (the same "train.csv" in Prob 5). In this problem, you will build up two classifiers: (i) the margin-based linear classifier; and (ii) the Least-Squares classifier (with a bias term). You will also compare the test error performances with test dataset $\{(x^{(i)}_{\text{test}}, y^{(i)}_{\text{test}})\}^{m_{\text{test}}}_{i=1}$ (given in "test.csv" uploaded). You need the write a script for Python implementation.

(b) Given the trained model $(a^*, b^*)$ in part (a), the margin-based linear classifier outputs:

$$ y_{\text {test }}^{(i)}=\operatorname{sign}\left(a^{* T} x_{\text {test }}^{(i)}-b^*\right)$$

Using a Python code below, compute the test error of the margin-based linear classifier:

```python 
import pandas as pd
import numpy as np

test = pd.read_csv('test.csv')
Xtest = test[['X1', 'X2']].values
ytest = test['Y'].values
m_test, n = Xtest.shape

error_LP=np.sign(Xtest@a.value - b.value) != ytest
print("Test error of the MB linear classifier: ",
        sum(error_LP)/m_test)
```

(c) Formulate an optimization for the least-squares classifier. Using a Python code below, develop the LS classifier and print out $(w^*, c^*)$.

```python 
import pandas as pd
import numpy as np
from numpy.linalg import pinv

data = pd.read_csv('train.csv')
X = data[['X1', 'X2']].values
y = data['Y'].values
m,n = X.shape

allone = np.ones((m,1))
A = np.concatenate((X,allone),1)

# closed form solution
w_bar_star = np.dot(pinv(A),y)

print('w*:', w_bar_star[0:2])
print('c*:', w_bar_star[2])
```

(d) Double-check your answer in part (c) using the following CVXPY script:

```python 
import pandas as pd
import cvxpy as cp
import numpy as np

data = pd.read_csv('train.csv')
X = data[['X1', 'X2']].values
y = data['Y'].values
m,n = X.shape

w = cp.Variable(n)
c = cp.Variable()

# constraints and the objective function
objective = cp.Minimize( cp.sum((X@w+c -y)**2) )
prob = cp.Problem(objective)
prob.solve()

print('status: ', prob.status)
print('optimal value: ', prob.value)
print('optimal var: ')
print('w*: ',w.value)
print('c*: ',c.value)
```

(e) Given the trained model $\left(w^*, c^*\right)$ in part $(c)$, the LS classifier outputs:
$$
y_{\text {test }}^{(i)}=\operatorname{sign}\left(w^{* T} x_{\text {test }}^{(i)}+c^*\right)
$$
Using a Python code, compute the test error of the LS classifier. Which classifier is better between the LS classifier and the margin-based linear classifier?